# Welcome to the Day 2 Lab!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get started --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## First - let's talk about the Chat Completions API

1. The simplest way to call an LLM
2. It's called Chat Completions because it's saying: "here is a conversation, please predict what should come next"
3. The Chat Completions API was invented by OpenAI, but it's so popular that everybody uses it!

### We will start by calling OpenAI again - but don't worry non-OpenAI people, your time is coming!


In [19]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


## Do you know what an Endpoint is?

If not, please review the Technical Foundations guide in the guides folder

And, here is an endpoint that might interest you...

In [20]:
import requests

headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

payload = {
    "model": "gpt-5-nano",
    "messages": [
        {"role": "user", "content": "Tell me a fun fact"}]
}

payload

{'model': 'gpt-5-nano',
 'messages': [{'role': 'user', 'content': 'Tell me a fun fact'}]}

In [22]:
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=payload
)

response.json()

{'id': 'chatcmpl-DBgTLPCaEbPTgwkAU8xFv5I1O10Ut',
 'object': 'chat.completion',
 'created': 1771677243,
 'model': 'gpt-5-nano-2025-08-07',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Fun fact: On Venus, a day is longer than a year. It takes about 243 Earth days to rotate once, but only about 224.7 days to orbit the Sun. Plus, Venus rotates in the opposite direction, so the Sun rises in the west.',
    'refusal': None,
    'annotations': []},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 11,
  'completion_tokens': 704,
  'total_tokens': 715,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 640,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': None}

In [23]:
response.json()["choices"][0]["message"]["content"]

'Fun fact: On Venus, a day is longer than a year. It takes about 243 Earth days to rotate once, but only about 224.7 days to orbit the Sun. Plus, Venus rotates in the opposite direction, so the Sun rises in the west.'

# What is the openai package?

It's known as a Python Client Library.

It's nothing more than a wrapper around making this exact call to the http endpoint.

It just allows you to work with nice Python code instead of messing around with janky json objects.

But that's it. It's open-source and lightweight. Some people think it contains OpenAI model code - it doesn't!


In [24]:
# Create OpenAI client

from openai import OpenAI
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content



'Wombat poop is cube-shaped. It’s one of the few animals that produces cubed feces, which helps it stack and not roll away, making it easier for wombats to mark their territory. Want another fun fact?'

## And then this great thing happened:

OpenAI's Chat Completions API was so popular, that the other model providers created endpoints that are identical.

They are known as the "OpenAI Compatible Endpoints".

For example, google made one here: https://generativelanguage.googleapis.com/v1beta/openai/

And OpenAI decided to be kind: they said, hey, you can just use the same client library that we made for GPT. We'll allow you to specify a different endpoint URL and a different key, to use another provider.

So you can use:

```python
gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key="AIz....")
gemini.chat.completions.create(...)
```

And to be clear - even though OpenAI is in the code, we're only using this lightweight python client library to call the endpoint - there's no OpenAI model involved here.

If you're confused, please review Guide 9 in the Guides folder!

And now let's try it!

## THIS IS OPTIONAL - but if you wish to try out Google Gemini, please visit:

https://aistudio.google.com/

And set up your API key at

https://aistudio.google.com/api-keys

And then add your key to the `.env` file, being sure to Save the .env file after you change it:

`GOOGLE_API_KEY=AIz...`


In [25]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

load_dotenv(override=True)

google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")



An API key was found, but it doesn't start AIz


In [26]:
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

response = gemini.chat.completions.create(model="gemini-2.5-flash-lite", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

BadRequestError: Error code: 400 - [{'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}]

## And Ollama also gives an OpenAI compatible endpoint

...and it's on your local machine!

If the next cell doesn't print "Ollama is running" then please open a terminal and run `ollama serve`

In [27]:
requests.get("http://localhost:11434").content

b'Ollama is running'

### Download llama3.2 from meta

Change this to llama3.2:1b if your computer is smaller.

Don't use llama3.3 or llama4! They are too big for your computer..

In [28]:
!ollama pull llama3.2:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 74701a8c35f6: 100% ▕██████████████████▏ 1.3 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 4f659a1e86d7: 100% ▕██████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [29]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')


In [30]:
# Get a fun fact

response = ollama.chat.completions.create(model="llama3.2:1b", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

"Here's a fun fact: Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible. This is because honey is one of the few foods that is self-preserving, meaning it doesn't require oxygen to stay fresh."

In [20]:
# Now let's try deepseek-r1:1.5b - this is DeepSeek "distilled" into Qwen from Alibaba Cloud

!ollama pull deepseek-r1:1.5b

python(19691) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling aabd4debf0c8: 100% ▕██████████████████▏ 1.1 GB                         
pulling c5ad996bda6e: 100% ▕██████████████████▏  556 B                         
pulling 6e4c38e1172f: 100% ▕██████████████████▏ 1.1 KB                         
pulling f4d24e9138dd: 100% ▕██████████████████▏  148 B                         
pulling a85fe2a2e58e: 100% ▕██████████████████▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [21]:
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

'<think>\nOkay, so I need to come up with a fun fact. Hmm, let\'s see... The user already got "The square root of 2 isn\'t irrational." That was interesting because it\'s something that often comes up in math. But maybe there are other things someone could find fun about.\n\nWait, another one I remember is the speed limit on a wooden ruler. So, if you try to write \'ruler\' on one end and \'width\' on the other, the line where they overlap can\'t be over 120 mph because that\'s too fast for a human. That sounds like something I could explain.\n\nLet me think about another fun fact. How about something related to technology? Maybe something as basic but impactful... Oh! The first practical computer keyboard was around 1963. It was really cool back then, especially since it was used in teaching. That\'s a good point because it shows how early technology shaped people.\n\nOr maybe something less techy, like cooking. I\'ve heard that corn on the cob isn\'t just corn; it has a lot of histor

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [31]:
from bs4 import BeautifulSoup
import requests


def fetch_website_contents(url):

    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
    }
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:100_000]
    



In [16]:
system_prompt = """ You are a smart Noun Assistant. and your duty is to answer any questions students have regarding NOUN based on the website.
"""

user_prompt_prefix = """
what is the requirement for bsc. economics?
"""
def getAnswer(question):
    website = fetch_website_contents("https://nou.edu.ng")
    response = ollama.chat.completions.create(
        model="llama3.2:1b",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "system", "content": "website: " + website},
            {"role": "user", "content":  question}
        ]
    )
    print(response.choices[0].message.content)
    return response.choices[0].message.content


In [33]:
ans = getAnswer("what is the total school fee for bsc. economics?")

print(ans)

According to the National Open University of Nigeria (NOUN) tuition fees, here is a breakdown of the estimated total semester school fees for BS/CAM (or BS Economics) courses:

**Please note that these fees are subject to change and may vary depending on individual circumstances**

For BS Economics (with options in Microeconomics and/or Macroeconomics):

* 1st year:
 + University Charges: ₦6,000,000 - ₦8,000,000 [approximately $15,000 - $22,000 USD]
 + NOUN Staffing charges: ₦800,000 – ₦1,200,000 (approximately $2,000 - $3,500 USD)
Total 1st year fees: ₦7,800,000 - ₦11,200,000 (approximately $19,700 - $30,500 USD)

For BS/CM (or BS Economics with combined courses):

* 1st semester:
 + University Charges:₦6,000,000-₦8,000,000 [approximately $15,000-$22,000 USD]
+ NOUN Staffing charges:₦800,000 – ₦1,200,000 (approximately $2,000-$3,500 USD)
Total 1st semester fees: ₦7,800,000-₦10,200,000 [approximately $19,700-$29,100 USD]

For BS Economics with specific modules:

* For example:
	+ Modul